# Interactive Data Visualization
##### (C) 2023-2025 Timothy James Becker: [revision 1.0](),  [GPLv3 license](https://www.gnu.org/licenses/gpl-3.0.html) 

## <u>Spatial Visualization</u>

Some types of data are inherently spatial and involve space as a primary component. As such they lend themself to visualization and work best in that context.  Some examples are geographic boundaries, roads, waterways and other such structures.  We will learn about geometric objects (points, lines and polygons) as well as the specialized version of JSON called GEOJSON that can describe these objects and the underlying tables (or ids) that are associated with them.  We will then look at using open-source map servers that provide REST services for drawing background geographic structures like state and town lines as well as roads and waterways.  We will finish our coverage by looking at a [GEOJSON](https://geojson.org/) data file from the CT that has county boundaries and then join it to a CSV table that has census records associated with it all over a [OpenStreetMap](https://www.openstreetmap.org) background that will show roads, waterways and even some building structures such as shopping, schools and hospitals.


#### Geometry Overview

We have points, lines and polygons in which we can draw relationships and overlay on any mapping of space. An important consideration is how these geometruc objects are drawn as shown below. Typically drawing will have a line or path as a series of ordered spatial coordinates. The lines or paths are drawn in order.  Polygons on the other can be more complex and are typically drawn in GEOJSON (at least for outershapes) in clockwide ordering (where inner holes in the polygon are drawn clockwise).

Lines (or paths are drawn as a series of ordered coordinates in the space). Here we can see that the lines have been drawn from left to right with increasing point numbers:

<img src="figures/geometry_lines_figure.png" alt="geometry lines" width="800px">

Polygons are more complex since the last line will connect the very first ordered point to the very last ordered point as shown below. Notice that a Polygon would be a line if the last closure line was left out.  Once polygons have been completed (byt the closer) additional holes can be carved out of the space inside (for GEOSJON those holes get drawn clockwise):

<img src="figures/geometry_polygons_figure.png" alt="geometry polygons" width="800px">

#### Geographic Coordinates

Geometric objects can be connected to individual rows of data and are typically drawn as points, lines (path in SVG) or polygons.  Previously our objects such as circles have existed in the domain of the columns of data we select and are mapped to our range which is the pixel space.  Since we are now display data that is inherently spatial, we will simply use the mapping space (known as a projection) to go directly to the pixel space. For geographic maps that inherent space has the units known as [longitude and latitude]( https://en.wikipedia.org/wiki/Geographic_coordinate_system).

<img style="padding:30px" src="https://upload.wikimedia.org/wikipedia/commons/e/ef/FedStats_Lat_long.svg" alt="longitude and latitude" width="500px"> <img style="padding:30px" src="https://upload.wikimedia.org/wikipedia/commons/0/0b/Solstice_revolving_earth.gif" alt="Equator" width="390px">

Latitude essentially is a circular unit in degrees that allows for +-90 degrees from the center point which is set at the [Equator](https://en.wikipedia.org/wiki/Equator) which is also the rotational band for the spinning earth surface as shown below in the GIF. When you exceed the 90-degree North (+90) you actually go back to < 90 degrees since this is a measure from 0 as shown in the figure above.  Longitude on the other hand measures degrees from East to West using a central point called the Prime Meridian (Greenwich). Interestingly Longitude doesn't have a magnetic or naturally occurring feature, but was devised as a reference because of shipping at sea [UK and US were dominant trade partners in 1880's when this was decided](https://en.wikipedia.org/wiki/Meridian_(geography)).


#### OpenStreetMap

This is a static embedded image view of openstreetmaps if you navigate in your browser:

<img src="figures/openstreetmaps_figure.png" alt="open_street_map" width="800px">

#### Interactive Geographic Visualization (Leaflet)

We you use the python embedded version of Leaflet here inside the jupyter notebook (python cells) to illustrate the initial ideas of Leaflet. After this brief introduction however, it will be better to make use of the [d3_leaflet_template_webapp](https://github.com/timothyjamesbecker/Interactive_Data_Visualization/tree/main/d3_leaflet_template_webapp) which is best with the webstorm IDE. The [folium](https://python-visualization.github.io/folium/latest/) library privides a nice mechanism for working with OpenStreetMaps and can be used along with other python data analysis tools such as pandas/geopandas, etc for more involved GIS processing.  Here in our work we will use preprocessed GIS files that will be easier to work with since GIS is typically a specialized and farily involved course on its own.

In [19]:
#run the installer line below if you want to try and istall the python embedded leaflet.js package called folium
#!pip install folium
import folium
map_osm = folium.Map(location=[41.39125,-72.10679],zoom_start=10)
map_osm

#### Loading (OpenStreetMaps) Maps

Starting with the leafletjs.org [quickstart guide](https://leafletjs.com/examples/quick-start/) we can see that we need both the leaflet.css file as well as the leaflet.js file. You should make your HTML like the template to get started, keeping in mind you will need to have the leaflet library (dist folder) with several files in order to build these types of webapps.

#### index.html will look like:

#### main.js file will look like:

In [ ]:
var pos = [41.38016733657364, -72.10705729845692]
var zoom
var map = L.map('map').setView(pos, zoom); //new leaflet map
L.tileLayer('https://tile.openstreetmap.org/{z}/{x}/{y}.png', {
    maxZoom: 13,
    attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
}).addTo(map);

#### The resulting app (once loaded) will look like:

<img src="figures/webstorm_leaflet_base.png" alt="leaflet_base" width="800px">

#### Drawing objects onto the Leaflet OpenStreetMap

Next, we will see that we can draw any of the basic geometric objects over the base tiles. In addition to a non-interactive object like a circle, we can also place interactive markers and use popup windows which are interactive visual items that can be manipulated in the geographic spatial dimension (Latitude/Longitude). They are useful for tagging points of interest or allowing a user to get more information about a location.

#### changing the main.js to draw a circle and a popup:

In [ ]:
var pos = [41.38016733657364, -72.10705729845692]
var zoom
var map = L.map('map').setView(pos, zoom); //new leaflet map
L.tileLayer('https://tile.openstreetmap.org/{z}/{x}/{y}.png', {
    maxZoom: 13,
    attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
}).addTo(map);

var marker = L.marker(pos).addTo(map); //new marker

var circle = L.circle([41.37893838463353, -72.10563452868446], { //draw a circle a little way away
    color: 'red',
    fillColor: '#f03',
    fillOpacity: 0.5,
    radius: 50
}).addTo(map);

#### The results will then look like:
<img src="figures/webstorm_leaflet_marker.png" alt="leaflet_marker" width="800px">

#### Drawing with GEOJSON files

Next we will load a GEOJSON file which has polygons for town boundaries on our map.  The GEOJSON file will become a javascript object once we load it with the d3 data loader (similar to the CSV/TSV table loader). We will then have a file that looks like:

Looking at the "geometry" key, we can see that a "Polygon" type has been specified with 19 points or "coordinates". Data that is associated with the geometric object can be placed inside under the "properties" key.  Here we have a few important parts which are the town name, its unique identifier (geoid for joining to a CSV data table), and the center x and center y values which are the Latitude and Longitude centroids of the polygon (AKA center of the town).

#### Joining Data to GEOJSON objects

The last component on this section that deals with spatial visualization will be to join a CSV data table to our polygons that are stored in the GEOJSON file. To join our data, we have to load multiple files. Since the d3 data loader that we have structured previously opens only one file and typically uses a then statement to bind a callback function to the request, we can simply nest our loaders so one then will have the second call. Alternatively, we can save the result of the d3 data loader (which is a promise) and then wait for them all to finish before making a single callback.


In [ ]:
d3.json("ct-towns-2022-simple-datactgov.geojson").then((geojson,err1)=> {
    // console.log(err1);
    // console.log(geojson);
    d3.dsv(",","ct-town-pop-density-2019acs5yr.csv",(d)=>{
        return{
            name:  d.Town,
            pop:  +d.pop2019,
            dense:+d.popsqmi2019
        };
    }).then((data,err2)=>{
        //area where both data are, now I can join, viz..
        // console.log(err2);
        // console.log(data);

        for(var i=0; i<geojson.features.length; i++){
            for(var j=0; j<data.length; j++){
                if(geojson.features[i].properties["name"]==data[j]["name"]){
                    geojson.features[i].properties["pop"]   = data[j]["pop"];
                    geojson.features[i].properties["dense"] = data[j]["dense"];
                }
            }
        }
        console.log(geojson);
        //leaflet here...
        var map = L.map('map').setView([41.38016733657364, -72.10705729845692], 12); //new leaflet map
        L.tileLayer('https://tile.openstreetmap.org/{z}/{x}/{y}.png', {
            maxZoom: 18,
            attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
        }).addTo(map);

        var marker = L.marker([41.37990540265118, -72.10761677627399]).addTo(map);
        var circle = L.circle([41.37893838463353, -72.10563452868446], {
            color: 'red',
            fillColor: '#f03',
            fillOpacity: 0.5,
            radius: 50
        }).addTo(map);

        var town_style = {
            "color": "#ff7800",
            "weight": 5,
            "opacity": 0.65
        };

        L.geoJSON(geojson,{
            style:town_style
        }).addTo(map);
});

Notice how we give the first dataset the name geojson and the second we name data. Using a for loop, we examine each polygon in the geojson object and try to match its property "name" to the corresponding data element "name". This works and looks so easy inside the then statement because in the data loader we took the time to use projection (renaming a column):

In [ ]:
return{
    name:  d.Town,
    pop:  +d.pop2019,
    dense:+d.popsqmi2019
};

#### The results of loading and drawing a GEOJSON will then look like:

<img src="figures/webstorm_leaflet_geojson.png" alt="leaflet_geojson" width="800px">

#### Making a Choropleth

A Choropleth is a special type of heatmap that uses preexisting spatial boundaries to compute the counts.  Some common examples are to look at various country, state, town or census tracks (or even smaller at block level) for relationships about those regions. We will take a CT state census data table that has population information and use it to color the spatial town boundaries, thereby making our first Choropleth. We basically need to modify the town_style section of code and use a function to change color dynamically instead of leaving it the same hex value.

In [ ]:
var dense_scale = d3.scaleLinear([10,9000],[0.1,1.0]);
d3.json("ct-towns-2022-simple-datactgov.geojson").then((geojson,err1)=> {
    d3.dsv(",","ct-town-pop-density-2019acs5yr.csv",(d)=>{
        return{ name:  d.Town, dense:+d.popsqmi2019 };
    }).then((data,err2)=>{
        for(var i=0; i<geojson.features.length; i++){
            for(var j=0; j<data.length; j++){
                if(geojson.features[i].properties["name"]  == data[j]["name"]){
                    geojson.features[i].properties["pop"]   = data[j]["pop"];
                    geojson.features[i].properties["dense"] = data[j]["dense"];
                }
            }
        }
        var map = L.map('map').setView([41.38016733657364, -72.10705729845692], 8); //new leaflet map
        L.tileLayer('https://tile.openstreetmap.org/{z}/{x}/{y}.png', {
            maxZoom: 18,
            attribution: '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a>'
        }).addTo(map);

        function town_style(feature){
            return {
                "color": "#0080ff",
                "weight": 1,
                "fillOpacity": dense_scale(feature.properties["dense"]),
            }
        }

        L.geoJSON(geojson,{style:town_style}).addTo(map);
    });
});

#### Which looks like this:

<img src="figures/webstorm_leaflet_choropleth.png" alt="leaflet_choropleth" width="800px">

Here we use a prebuilt d3 linear scale to change the amount of opacity in each polygon by looking inside the properties for the dense value. We map dense values of 10 to opacity of 0.1, while dense values of 9000 or higher will map to opacity of 1.0 (full color shown)

# Exercises

#### [1] Starting with a fresh [d3_leaflet_template_webapp](https://github.com/timothyjamesbecker/Interactive_Data_Visualization/tree/main/d3_leaflet_template_webapp) can you modify the fill color instead of the opacity?
#### [2] Can you change the opacity or color of each town using the population column instead of density?
#### [3] Can you add a mouse-over interaction like the [tutorial shows](https://leafletjs.com/examples/choropleth/)?